<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/vk_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vk_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 44.7 MB/s eta 0:00:00


In [ ]:
import os
import getpass
import csv
import pandas as pd
import vk_api
from github import Github

In [ ]:
os.environ["VK_API_TOKEN"] = getpass.getpass("Введите VK API Token:")

Введите VK API Token:··········


In [ ]:
os.environ["GITHUB_TOKEN"] = getpass.getpass("Введите Github Token:")

Введите Github Token:··········


In [ ]:
def get_vk_api(token):
    """Создает объект API для работы с VK."""
    vk_session = vk_api.VkApi(token=token)
    return vk_session.get_api()

def get_group_posts(group_id):
    """Получает все посты группы по domain или owner_id."""
    vk = get_vk_api(os.getenv("VK_API_TOKEN"))
    posts = []

    try:
        if group_id.isdigit():
            owner_id = f"-{group_id}"
        else:
            group_info = vk.groups.getById(group_id=group_id)
            owner_id = f"-{group_info[0]['id']}"

        response = vk.wall.get(owner_id=owner_id, count=100)
        posts.extend(response['items'])

        while 'next_from' in response:
            response = vk.wall.get(owner_id=owner_id, count=100, offset=len(posts))
            posts.extend(response['items'])

    except Exception as e:
        print(f"Error retrieving posts: {e}")

    return posts

def get_group_posts_with_comments(group_id):
    """Получает все посты группы с комментариями к ним."""
    vk = get_vk_api(os.getenv("VK_API_TOKEN"))
    posts_with_comments = []

    try:
        if group_id.isdigit():
            owner_id = f"-{group_id}"
        else:
            group_info = vk.groups.getById(group_id=group_id)
            owner_id = f"-{group_info[0]['id']}"

        response = vk.wall.get(owner_id=owner_id, count=100)

        for post in response['items']:
            comments = vk.wall.getComments(owner_id=owner_id, post_id=post['id'], count=100)
            posts_with_comments.append({
                'post': post,
                'comments': comments['items']
            })

        while 'next_from' in response:
            response = vk.wall.get(owner_id=owner_id, count=100, offset=len(posts_with_comments))
            for post in response['items']:
                comments = vk.wall.getComments(owner_id=owner_id, post_id=post['id'], count=100)
                posts_with_comments.append({
                    'post': post,
                    'comments': comments['items']
                })

    except Exception as e:
        print(f"Error retrieving posts with comments: {e}")

    return posts_with_comments

def get_group_subscribers(group_id):
    """Получает информацию о подписчиках группы."""
    vk = get_vk_api(os.getenv("VK_API_TOKEN"))
    subscribers = []

    try:
        if group_id.isdigit():
            owner_id = f"-{group_id}"
        else:
            group_info = vk.groups.getById(group_id=group_id)
            owner_id = f"{group_info[0]['id']}"

        subscribers_info = vk.groups.getMembers(group_id=owner_id, fields="id, first_name, last_name, sex, bdate, city, country")
        subscribers.extend(subscribers_info['items'])

    except Exception as e:
        print(f"Error retrieving subscribers: {e}")

    return subscribers

def save_to_csv(data, filename):
    """Сохраняет данные в CSV файл."""
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

def upload_to_github(file_path, func):
    """Загружает CSV файл в репозиторий на GitHub в папку data/vk."""
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo("componavt/sns4human")

    if func == "post":
      filename = os.path.basename(file_path)
      target_path = f"data/vk/posts/{filename}"

    if func == "sub":
      filename = os.path.basename(file_path)
      target_path = f"data/vk/followers/{filename}"

    with open(file_path, "rb") as file:
        content = file.read()

    try:
        # Проверяем, существует ли файл по целевому пути
        repo.get_contents(target_path)
        # Если существует, обновляем его
        repo.update_file(target_path, "Update " + target_path, content, repo.get_contents(target_path).sha)
        print(f"Файл {target_path} успешно обновлен.")
    except Exception as e:
        # Если не существует, создаем новый файл
        repo.create_file(target_path, "Add " + target_path, content)
        print(f"Файл {target_path} успешно добавлен.")


def get_group_name(group_id_or_domain):
    """
    Получает название группы ВКонтакте по ID или домену.

    :param group_id_or_domain: ID группы или её домен (например, 'mygroup').
    :return: Название группы или None, если группа не найдена.
    """

    vk = get_vk_api(os.getenv("VK_API_TOKEN"))

    try:
        # Запрос информации о группе
        group_info = vk.groups.getById(group_id=group_id_or_domain)
        return group_info[0]['name']
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def transliterate_russian_to_english(text):
    # Словарь для замены русских букв на английские
    transliteration_dict = {
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd',
        'е': 'e', 'ё': 'yo', 'ж': 'zh', 'з': 'z', 'и': 'i',
        'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n',
        'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't',
        'у': 'u', 'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch',
        'ш': 'sh', 'щ': 'shch', 'ъ': '', 'ы': 'y', 'ь': '',
        'э': 'e', 'ю': 'yu', 'я': 'ya'
    }

    # Заменяем русские буквы на английские
    transliterated_text = ''.join(transliteration_dict.get(c, c) for c in text.lower())

    # Заменяем пробелы на подчеркивания
    safe_text = transliterated_text.replace(' ', '_')

    return safe_text

**Группы и их домены:**

Православие в Карелии: pravkarelia

Православные Карелии: club2975745

Прихожане Крестовоздвиженского собора: krest_sobor

Александро-Невский собор г.Петрозаводска: nevsoborptz

Приход церкви св.вмц Екатерины: ekaterinahram

Прихожане Сретенского храма: club19347481

Монастырь Александра Свирского: svirskiymonastery

Подворье Соловецкого монастыря в Кеми (п.Раб-ск): kemskoepodvorie

Лютеране России — Церковь Ингрии: infoinkeri

Кемский лютеранский приход: kemskij_prihod

Кондопожский евангелическо-лютеранский приход: club18959947


In [ ]:
group_id = input("Введите ID или domain группы: ")
group_name = get_group_name(group_id)
safe_group_name = ''.join(c for c in group_name if c.isalnum() or c in (' ', '_')).rstrip()
safe_group_name = transliterate_russian_to_english(safe_group_name)

    # Получаем посты группы
posts = get_group_posts(group_id)
save_to_csv(posts, f"{safe_group_name}.csv")

    # Получаем посты с комментариями
#posts_with_comments = get_group_posts_with_comments(group_id)
#save_to_csv(posts_with_comments, f"{safe_group_name}_group_posts_with_comments.csv")

    # Получаем подписчиков группы
#subscribers = get_group_subscribers(group_id)
#save_to_csv(subscribers, f"{safe_group_name}_group_subscribers.csv")

# Загружаем файлы на GitHub
upload_to_github(f"{safe_group_name}.csv", "post")
#upload_to_github(f"{safe_group_name}_group_posts_with_comments.csv")
#upload_to_github(f"{safe_group_name}_group_subscribers.csv")

Введите ID или domain группы: pravkarelia
Файл data/vk/posts/pravoslavie_v_karelii.csv успешно добавлен.


Получение постов группы

In [ ]:
group_id = input("Введите ID или domain группы: ")
group_name = get_group_name(group_id)
safe_group_name = ''.join(c for c in group_name if c.isalnum() or c in (' ', '_')).rstrip()
safe_group_name = transliterate_russian_to_english(safe_group_name)
posts = get_group_posts(group_id)
save_to_csv(posts, f"{safe_group_name}.csv")
upload_to_github(f"{safe_group_name}.csv", "post")

Получение информации о подписчиках группы

In [ ]:
group_id = input("Введите ID или domain группы: ")
group_name = get_group_name(group_id)
safe_group_name = ''.join(c for c in group_name if c.isalnum() or c in (' ', '_')).rstrip()
safe_group_name = transliterate_russian_to_english(safe_group_name)
subscribers = get_group_subscribers(group_id)
save_to_csv(subscribers, f"{safe_group_name}.csv")
upload_to_github(f"{safe_group_name}.csv", "sub")